<a href="https://colab.research.google.com/github/leochartrand/IFT615/blob/main/RNN/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Apprenons à un RNN à générer du Shakespeare!

*Mary, and will, my lord, to weep in such a one were prettiest​*

​*Yet now I was adopted heir​​*

​*Of the world’s lamentable day​​*

​*To watch the next way with his father with his face?*

### Dans ce notebook, nous survolerons une implémentation simple d'une réseau de neurones récurrent qui apprend, lettre par lettre, à générer du texte ressemblant à du Shakespeare. Le développement d'un RNN étant beaucoup plus complexe que pour des réseaux de neurones simples ou à convolution, plusieurs concepts ne seront pas abordés pour qu'on puisse se concentrer sur le fonctionnement général. Dans cette démonstration, vous apprendrez à quoi ressemble le traitement de données de texte afin qu'il soit interpreté par un modèle lors de son apprentissage. Nous passerons ensuite en revue une architecture de RNN simple ainsi que son entraînement.

Commençons par importer nos librairies

In [110]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

Téléchargeons maintenant le jeu de données. Ce dernier est un ensemble de 40,000 lignes de Shakespeare provenant d'une variété de ses oeuvres. Ce *dataset* a été assemblé par Andrej Karpathy: https://github.com/karpathy/char-rnn (BSD License)

In [ ]:
!wget https://raw.githubusercontent.com/leochartrand/IFT615/main/RNN/shakespeare.txt

data = open(r'shakespeare.txt').read()

Jetons un coup d'oeil au dataset. On peut imprimer par exemple les 1000 premiers charactères:

In [ ]:
print(data[:1000])

Prétraitons le jeu de donner un peu plus. Commençons par créer une "dictionnaire" des caractères que l'on trouve dans le jeu de données:

In [ ]:
char_dict = sorted(list(set(data)))
vocab_size = len(char_dict)
print("Le jeu de données est constitué de %d caractères uniques."%vocab_size)

Il est important que les caractères puissent être interprétés par notre modèle. Pour cela, on peut commencer par convertir chaque charactère en un indice, qui prend position dans l'intervalle [0, vocab_size[. On veut aussi pouvoir faire l'inverse et obtenir des caractères à nouveau à la sortie du modèle:

In [50]:
# Pour éviter de faire la conversion plusieurs fois et de répéter du code, on
# peut créer des dictionnaires qui font un "mapping" entre les caractères et leur index
char_to_index = {char:i for i,char in enumerate(char_dict)}
index_to_char = {i:char for i,char in enumerate(char_dict)}

# Créons une simple fonction qui permettra d'obtenir les caractères directement lors de la génération:
def get_char(index):
  return index_to_char[index]
def get_index(char):
  return char_to_index[char]

# On peut déjà convertir notre jeu de données en une liste d'indices!
data = list(data)
for i, char in enumerate(data):
    data[i] = get_index(char)

On peut maintenant créer notre modèle! On peut voir ici que le modèle comporte en fait 3 sous-modules:
 - Une couche de "embedding": Cette couche permet de projeter les caractères dans un espace latent où les relations entre ces derniers peuvent être établies selon leur position dans l'espace. La sortie de cette couche est donc un vecteur.
 - Une couche RNN: Cette couche prend en entrée le vecteur de caractère et ainsi qu'un état caché (aussi un vecteur) qui représente tous les caractères ayant été vu avant celui-ci. La sortie de cette couche est un état caché mis à jour ainsi qu'un vecteur de sortie.
 - Une couche dense finale qui permet la classification. Elle prend en entrée le vecteur de prédiction sortant du RNN et le transforme en vecteur de prédictions pour chaque caractère.

 Mais lorsqu'on lit la première lettre, d'où vient l'état caché? En fait, il suffit de donner un tensor nul (à zéro), qui permettra de prendre que le caractère dans l'équation à ce moment. C'est à cela que sert la fonction init_hidden.

In [92]:
class Char_RNN(nn.Module):
  def __init__(self, input_size, output_size, hidden_size, num_layers):
    super(Char_RNN, self).__init__()
    self.layers = num_layers
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(input_size, input_size)
    self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
    self.dense = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    embedded = self.embedding(input)
    output, hidden = self.rnn(embedded, hidden)
    output = self.dense(output)
    return output, hidden

  def init_hidden(self, batch_size):
        return torch.zeros(self.layers, batch_size, self.hidden_size)

char_model = Char_RNN(vocab_size, vocab_size, 128, 3)

On passe à l'entraînement... Dans ce bloc de code, il y a beaucoup d'opérations qui ne seront pas expliquées dans le cadre de ce cours. Ce qu'il faut retenir, c'est que les données de texte sont de nature linéaire et de taille variable. Cela requiert un prétraitement de données un peu plus élaboré que ce qu'on a vu avec d'autres types de réseau.

Lors de l'entrainement, les données sont présentées en séquences d'un longueur prédéterminée. Le modèle prédit le caractère qui suit la séquence et compare avec la vérité terrain.

In [ ]:
# Définissons nos hyperparametres
epochs = 5
batch_size = 100
sequence_length = 50
learning_rate = 5e-4

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(char_model.parameters(), lr=learning_rate)

# On garde un historique de l'entraînement pour visualiser nos résultats
loss_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_batches = (len(data) - sequence_length) // (sequence_length * batch_size) # nombre de batches au total
    with tqdm(total=total_batches) as progress:
        for _ in range(total_batches):
            start_index = random.randint(0, 100)

            # Détecte si on est rendu à la dernière batch de l'epoch
            if start_index + sequence_length * batch_size > len(data) - sequence_length:
                continue

            # Génère les séquences utilisées comme entrées et les séquences cibles (un caractère plus loin)
            input_seq = torch.tensor([data[j:j+sequence_length] for j in range(start_index, start_index + sequence_length * batch_size, sequence_length)])
            target_seq = torch.tensor([data[j+1:j+sequence_length+1] for j in range(start_index, start_index + sequence_length * batch_size, sequence_length)])

            if input_seq.size(0) != batch_size:
                continue

            # Initialise l'état caché et prépare le modèle
            hidden = char_model.init_hidden(batch_size)
            char_model.zero_grad()

            # Forward pass
            output, hidden = char_model(input_seq, hidden)
            # Calcule de la perte
            loss = criterion(output.view(-1, vocab_size), target_seq.view(-1))

            # Rétropropagation des gradients
            loss.backward()
            optimizer.step()

            # Pour visualisation de l'apprentissage
            loss_history.append(loss.item())
            progress.update(1)
            progress.set_description("Loss: %.4f"%loss.item())

Visualisons la courbe d'apprentissage:

In [ ]:
plt.plot(loss_history)
plt.xlabel('Batches')
plt.ylabel('Loss')
plt.title('Loss history')
plt.show()

### Essayons maintenant degénérer du Shakespeare!

In [ ]:
seed = "Bob"        # Bout de texte à partir duquel le modèle commence la génération
temperature = 0.9   # Hyperparamètre qui permet une stochasticité de génération, i.e. ne pas toujours obtenir le même texte
gen_length = 2000   # Longueur du texte généré (en caractères)

char_model.eval()

with torch.no_grad():
  for _ in range(gen_length):
    # On prépare les données et le modèle
    input_tensor = torch.tensor([[get_index(c) for c in seed][-sequence_length:]], dtype=torch.long)
    hidden = char_model.init_hidden(1)

    # Forward pass
    output, hidden = char_model(input_tensor, hidden)

    # On sélectionne le charactère selon les probabilités (sous forme d'indice)
    # Ici, une variable de température assure une certaine stochasticité
    logits = output[0, -1, :] / temperature
    # On normalise les probabilités avec un softmax
    prob_dist = F.softmax(logits, dim=0)

    # On échantillone ensuite un indice dans la distribution
    top_char_index = torch.multinomial(prob_dist, 1).item()

    # On reconvertit en charactère
    predicted_char = get_char(top_char_index)
    seed += predicted_char

    # Print the new character
    print(predicted_char, end='', flush=True)


On peut voir qu'en très peut de temps, un modèle RNN peut apprendre à générér du texte relativement vraisemblable et qui peut adopter un style particulier. Remarquez qu'ici, le modèle travaille avec les caractères plutôt qu'avec les mots. Si on avait choisi ces derniers, cela aurait plusieurs effets. Premièrement, le vocabulaire deviendrait immense et donc de même que pour la couche d'embedding. Cependant, la cohérence des phrases serait bien meilleure étant donné que le modèle établirait une relation entre les mots directement. En général, un modèle de langage travaille surtout avec les mots.